In [1]:
try:
    import qqdm
except:
    ! pip install qqdm > /dev/null 2>&1
print("Done!")
from PIL import Image
import csv
from IPython.display import display
import glob, random
from collections import OrderedDict

import numpy as np
import json

try:
    from qqdm.notebook import qqdm as tqdm
except ModuleNotFoundError:
    from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from IPython.display import display
import math
import os
import cv2
import matplotlib.pyplot as plt
import copy
import matplotlib.patches as patches
vgg16_conv = None

Done!


In [2]:
!nvidia-smi

Tue May 17 21:40:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 495.44       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:10:00.0 Off |                  Off |
| 30%   26C    P5    22W / 300W |      0MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 新段落

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# workspace_dir = '.'

In [5]:
# # # # # # Use `unzip' command to decompress
# !unzip '/content/drive/MyDrive/mini-imagenet (2).zip' -d '/content/miniImageNet'

In [6]:
class Vgg16Conv(nn.Module):  # conv搭建方法介绍：https://blog.csdn.net/qq_38607066/article/details/98517116

    def __init__(self, num_cls=1000):
        super(Vgg16Conv, self).__init__()

        self.features = nn.Sequential(  # 序贯模型 一个有序的容器，神经网络模块将按照在传入构造器的顺序依次被添加到计算图中执行，同时以神经网络模块为元素的有序字典也可以作为传入参数。
            # conv1
            nn.Conv2d(3, 64, 3, padding=1),  # input output kernal //padding=1,那么就会在原来输入层的基础上,上下左右各补一行
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True),  # return_indices：True-记录输入图片池化时最大值的位置，之后反卷积中可以使用

            # conv2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True),

            # conv3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True),

            # conv4
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True),

            # conv5
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True)
        )

        self.classifier = nn.Sequential(  # 全连接层 分类器
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_cls),
            nn.Softmax(dim=1)
        )

        self.conv_layer_indices = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]  # 卷积层所在层数
        self.feature_maps = OrderedDict()  # 有序字典，之后用于存储中间输出结果
        self.pool_locs = OrderedDict()  # 用于存储图片最大池化过程中最大值所在位置

        self.init_weights()  # 调用权重初始化函数，将VGG16模型的参数导入

    def init_weights(self):  # 将vgg16预训练模型的每层权重赋予此模型中（即自己搭建一个vgg16模型的框架，使用预训练模型的权重）
        vgg16_pretrained = models.vgg16(pretrained=True)
        # 卷积部分赋权值
        for idx, layer in enumerate(vgg16_pretrained.features):
            if isinstance(layer, nn.Conv2d):
                self.features[idx].weight.data = layer.weight.data  # 权重 weight
                self.features[idx].bias.data = layer.bias.data  # 偏置 bias
        # 全连接部分赋权值
        for idx, layer in enumerate(vgg16_pretrained.classifier):
            if isinstance(layer, nn.Linear):
                self.classifier[idx].weight.data = layer.weight.data
                self.classifier[idx].bias.data = layer.bias.data

    def check(self):
        model = models.vgg16(pretrained=True)
        return model

    def forward(self, x):  # 前向传播过程
        for idx, layer in enumerate(self.features):
            if isinstance(layer, nn.MaxPool2d):  # 在卷积部分的池化时，记录下池化过程中最大值位置
                x, location = layer(x)
            else:
                x = layer(x)

        x = x.view(x.size()[0], -1)  # 把数据size变成(1, 512 * 7 * 7)，将数据平铺成1维（相当于flatten层）# .view方法中，-1只有一个，表示该维度值自适应计算得

        output = self.classifier(x)  # 平铺数据后输入全连接层
        return output
    def conv_forward(self, x):  # 前向传播过程
        for idx, layer in enumerate(self.features):
            if isinstance(layer, nn.MaxPool2d):  # 在卷积部分的池化时，记录下池化过程中最大值位置
                x, location = layer(x)
            else:
                x = layer(x)

        # x = x.view(x.size()[0], -1)  # 把数据size变成(1, 512 * 7 * 7)，将数据平铺成1维（相当于flatten层）# .view方法中，-1只有一个，表示该维度值自适应计算得

        # output = self.classifier(x)  # 平铺数据后输入全连接层
        return x


In [7]:
workspace_dir = '.'
device = "cuda" if torch.cuda.is_available() else "cpu"

# fix random seeds
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)


In [8]:
def predict_conv(img):
    global vgg16_conv
    if vgg16_conv is None:
        vgg16_conv = Vgg16Conv()
        vgg16_conv.eval()
        vgg16_conv.to(device)
    return vgg16_conv.conv_forward(img)

In [9]:
def predict(img):
    global vgg16_conv
    if vgg16_conv is None:
        vgg16_conv = Vgg16Conv()
        vgg16_conv.eval()
        vgg16_conv.to(device)
    return vgg16_conv.forward(img)

In [10]:
class Classifier(nn.Module):
    def __init__(self, in_ch, k_way):
        super(Classifier, self).__init__()
        # vgg16_conv=Vgg16Conv()
        # self.vgg16_conv = vgg16_conv
        # self.vgg16_conv.eval()
        # self.vgg16_conv.to(device)
        # store(vgg16_conv)
        # self._initialize_weights()
        self.logits=nn.Linear(512*7*7, 5,bias=False)

    def forward(self, x):
        # x = self.vgg16_conv(x)
        # x = predict_conv(x)
        # x = self.logits(x)
        return predict(x)

    def functional_forward(self, x, params):
        '''
        Arguments:
        x: input images [batch, 1, 28, 28]
        params: model parameters, 
                i.e. weights and biases of convolution
                      and weights and biases of 
                                    batch normalization
                type is an OrderedDict

        Arguments:
        x: input images [batch, 1, 28, 28]
        params: The model parameters, 
                i.e. weights and biases of convolution 
                      and batch normalization layers
                It's an `OrderedDict`
        '''
        # self.vgg16_conv(x)
        x = predict_conv(x)
        x = x.view(x.shape[0], -1)
        x = F.linear(x,
                         params['logits.weight'])
                        #  params['logits.bias'])
        # return predict(x)
        return x

        # return logits

In [11]:
# class newClassifier(nn.Module):
#   def __init__(self, in_ch, k_way,prototype_shape,img_size,epsilon,prototype_activation_function):
#     super(newClassifier, self).__init__()
#     self.conv1 = ConvBlock(in_ch, 64)
#     self.conv2 = ConvBlock(64, 64)
#     self.conv3 = ConvBlock(64, 64)
#     self.conv4 = ConvBlock(64, 64)
#     # self.conv4 = ConvBlock4(64, 64)
# #         self.add_on_layers = nn.Sequential(
# #             nn.Conv2d(64,128,kernel_size=1),
# #             nn.ReLU(),
# #             nn.Conv2d(128,128,kernel_size=1),
# #             nn.Sigmoid()
# #         )
#     prototype_shape = prototype_shape
#     self.prototype_shape = prototype_shape
#     self.img_size = img_size
#     self.num_prototypes = prototype_shape[0]
#     self.num_classes = k_way
#     self.epsilon = epsilon
#     self.prototype_activation_function =  prototype_activation_function
#     assert(self.num_prototypes % self.num_classes == 0)
#     self.prototype_class_identity = torch.zeros(self.num_prototypes,
#                           self.num_classes)
#     num_prototypes_per_class = self.num_prototypes // self.num_classes
#     for j in range(self.num_prototypes):
#         self.prototype_class_identity[j, j // num_prototypes_per_class] = 1
        
#     self.prototype_vectors = nn.Parameter(torch.rand(self.prototype_shape),
#                                           requires_grad=True)
#     self.ones = nn.Parameter(torch.ones(self.prototype_shape),
#                               requires_grad=False)
#     self.logits = nn.Linear(self.num_prototypes, k_way
#                                 ,bias=False
#                             ) # do not use bias    
#     self._initialize_weights()
  
  
  
#   def forward(self, x):
#     x = self.conv1(x)
#     x = self.conv2(x)
#     x = self.conv3(x)
#     x = self.conv4(x)
#     x = x.view(x.shape[0], -1)
#     x = self.logits(x)
#     return x

  
#   def set_last_layer_incorrect_connection(self, incorrect_strength):
#     '''
#     the incorrect strength will be actual strength if -0.5 then input -0.5
#     '''
#     positive_one_weights_locations = torch.t(self.prototype_class_identity)
#     negative_one_weights_locations = 1 - positive_one_weights_locations

#     correct_class_connection = 1
#     incorrect_class_connection = incorrect_strength
#     self.logits.weight.data.copy_(
#         correct_class_connection * positive_one_weights_locations
#         + incorrect_class_connection * negative_one_weights_locations)

  
#   def functional_forward(self, x, params):
#     '''
#     Arguments:
#     x: input images [batch, 1, 28, 28]
#     params: model parameters, 
#             i.e. weights and biases of convolution
#                   and weights and biases of 
#                                 batch normalization
#             type is an OrderedDict

#     Arguments:
#     x: input images [batch, 1, 28, 28]
#     params: The model parameters, 
#             i.e. weights and biases of convolution 
#                   and batch normalization layers
#             It's an `OrderedDict`
#     '''
    
#     # for block in [1, 2,3,4]:
#     #     x = ConvBlockFunction(
#     #         x,
#     #         params[f'conv{block}.0.weight'],
#     #         params[f'conv{block}.0.bias'],
#     #         params.get(f'conv{block}.1.weight'),
#     #         params.get(f'conv{block}.1.bias'))
#     # for block in [4]:
#     #     x = ConvBlockFunction4(
#     #         x,
#     #         params[f'conv{block}.0.weight'],
#     #         params[f'conv{block}.0.bias'],
#     #         params.get(f'conv{block}.1.weight'),
#     #         params.get(f'conv{block}.1.bias'))

#     x2 = x **2
#     x2_patch_sum = F.conv2d(x,params['ones'])

#     p2 = params['prototype_vectors'] ** 2
#     p2 = torch.sum(p2, dim=(1, 2, 3))
#     p2_reshape = p2.view(-1, 1, 1)
#     xp = F.conv2d(x,params['prototype_vectors'])
#     intermediate_result = - 2 * xp + p2_reshape  # use broadcast
#     # x2_patch_sum and intermediate_result are of the same shape
#     distances = F.relu(x2_patch_sum + intermediate_result)
    
#     min_distances = -F.max_pool2d(-distances,
#                           kernel_size=(distances.size()[2],
#                                         distances.size()[3]))
#     min_distances = min_distances.view(-1, self.num_prototypes)
    
#     prototype_activations = torch.log((min_distances + 1) / (min_distances + self.epsilon))
#     logits = F.linear(prototype_activations,params['logits.weight'])
# #         x = x.view(x.shape[0], -1)
# #         x = F.linear(x,
# #                      params['logits.weight'])
# #                      params['logits.bias'])
#     return logits,min_distances
    
  
#   def _initialize_weights(self):
# #         for m in self.add_on_layers.modules():
# #             if isinstance(m, nn.Conv2d):
# #                 # every init technique has an underscore _ in the name
# #                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

# #                 if m.bias is not None:
# #                     nn.init.constant_(m.bias, 0)

# #             elif isinstance(m, nn.BatchNorm2d):
# #                 nn.init.constant_(m.weight, 1)
# #                 nn.init.constant_(m.bias, 0)
#     self.set_last_layer_incorrect_connection(incorrect_strength=-0.5)
  
  
#   def push_forward(self, x,params):
#     '''this method is needed for the pushing operation'''
#     for block in [1, 2, 3,4]:
#         x = ConvBlockFunction(
#             x,
#             params[f'conv{block}.0.weight'],
#             params[f'conv{block}.0.bias'],
#             params.get(f'conv{block}.1.weight'),
#             params.get(f'conv{block}.1.bias'))
#     # for block in [4]:
#     #     x = ConvBlockFunction4(
#     #         x,
#     #         params[f'conv{block}.0.weight'],
#     #         params[f'conv{block}.0.bias'],
#     #         params.get(f'conv{block}.1.weight'),
#     #         params.get(f'conv{block}.1.bias'))
#     conv_output=x
    
#     x2 = x ** 2
#     x2_patch_sum = F.conv2d(input=x2, weight=params['ones'])
#     p2 = params['prototype_vectors'] ** 2
#     p2 = torch.sum(p2, dim=(1, 2, 3))
#     # p2 is a vector of shape (num_prototypes,)
#     # then we reshape it to (num_prototypes, 1, 1)
#     p2_reshape = p2.view(-1, 1, 1)

#     xp = F.conv2d(input=x, weight=params['prototype_vectors'])
#     intermediate_result = - 2 * xp + p2_reshape  # use broadcast
#     # x2_patch_sum and intermediate_result are of the same shape
#     distances = F.relu(x2_patch_sum + intermediate_result)
#     return conv_output, distances

  


In [12]:
def create_label(n_way, k_shot):
    return (torch.arange(n_way)
                .repeat_interleave(k_shot)
                .long())

In [13]:
def calculate_accuracy(logits, val_label,n_way,q_query):
    """ utility function for accuracy calculation """
    acc = np.asarray([(
          torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy())]
          ).mean()
    precision =[0]*n_way
    recall = [0]*n_way
    f1 = [0]*n_way
    TFP=[0]*n_way
    TP=[0]*n_way
    pred = torch.argmax(logits, -1).cpu().numpy()
    ground = val_label.cpu().numpy()
    for index in range(len(ground)):
        for n_index in range(n_way):
            if pred[index]==n_index:
                TFP[n_index]=TFP[n_index]+1
                if ground[index]==n_index:
                    TP[n_index] = TP[n_index]+1
        for n_index in range(n_way):
            if TP[n_index] == 0:
                precision[n_index]=0
                recall[n_index]=0
                f1[n_index]=0
            else:
                precision[n_index] = TP[n_index]/TFP[n_index]
                recall[n_index] = TP[n_index]/q_query
                f1[n_index] = 2*precision[n_index]*recall[n_index]/(precision[n_index]+recall[n_index])


    return acc,np.array(precision).mean(),np.array(recall).mean(),np.array(f1).mean()


In [14]:
def predict_true(logits, val_label):
    """ utility function for accuracy calculation """
    acc = np.asarray([(
      torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy())]
      )
    acc= acc[0]
    res=[]
    for x in acc:
        res =res+[bool(x)]
    return res

In [15]:
class MiniImageNet(Dataset):
    def __init__(self, data_dir,k_way,q_query):
        """Dataset class representing miniImageNet dataset

        # Arguments:
            subset: Whether the dataset represents the background or evaluation set
        """
        self.file_list = [f for f in glob.glob(
            data_dir+"*", 
            recursive=True)]
        self.n = k_way + q_query        

        normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                  std=(0.229, 0.224, 0.225))
        # Setup transforms
        self.transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
            normalize,
        ])

    def __getitem__(self, idx):
        sample = np.arange(600)

        # For random sampling the characters we want.
        np.random.shuffle(sample)
        img_path = self.file_list[idx]
        img_list = [f for f in glob.glob(
            img_path + "**/*.jpg", recursive=True)]
        img_list.sort()
        imgs = [self.transform(
            Image.open(img_file)) 
            for img_file in img_list]
        # `k_way + q_query` examples for each character
        imgs = torch.stack(imgs)[sample[:self.n]]
        lst = [img_path.split('/')[-1]]+[imgs]
        return lst

    def __len__(self):
        return len(self.file_list)

In [16]:
def inner_update_MAML(fast_weights, loss, inner_lr):
    """ Inner Loop Update """
    grads = torch.autograd.grad(
        loss, fast_weights.values(), create_graph=True)
    # Perform SGD
    fast_weights = OrderedDict(
        (name, param - inner_lr * grad)
        for ((name, param), grad) in zip(fast_weights.items(), grads))
    return fast_weights

In [17]:
def collect_gradients_MAML(
    special_grad: OrderedDict, fast_weights, model, len_data):
    """ Actually do nothing (just backwards later) """
    return special_grad

In [18]:
def outer_update_MAML(model, meta_batch_loss, grad_tensors):
    """ Simply backwards """
    meta_batch_loss.backward()

In [19]:
def dataloader_init(datasets,n_way,num_workers=4):
    train_set, val_set, test_set = datasets
    train_loader = DataLoader(train_set,
                # The "batch_size" here is not \
                #    the meta batch size, but  \
                #    how many different        \
                #    characters in a task,     \
                #    i.e. the "n_way" in       \
                #    few-shot classification.
                batch_size=n_way,
                num_workers=num_workers,
                shuffle=True,
                drop_last=True)
    val_loader = DataLoader(val_set,
              batch_size=n_way,
              num_workers=num_workers,
              shuffle=True,
              drop_last=True)
    test_loader = DataLoader(test_set,
              batch_size=n_way,
              num_workers=num_workers,
              shuffle=True,
              drop_last=True)
    train_iter = iter(train_loader)
    val_iter = iter(val_loader)
    test_iter = iter(test_loader)
    return (train_loader, val_loader, test_loader), \
          (train_iter, val_iter, test_iter)



In [20]:
def model_init(n_way,in_ch,meta_lr):
    meta_model = Classifier(in_ch, n_way).to(device)
    optimizer = torch.optim.Adam(meta_model.parameters(), 
                                lr=meta_lr)
    loss_fn = nn.CrossEntropyLoss().to(device)
    # return meta_model, False, loss_fn
    return meta_model,optimizer,loss_fn



In [21]:
# def model_init_new(n_way,in_ch,meta_lr,prototype_shape,img_size,epsilon,prototype_activation_function):
#   meta_multi = newClassifier(in_ch, n_way,prototype_shape,img_size,epsilon,prototype_activation_function).to(device)
#   joint_optimizer_lrs = {'features': 1e-4,# 有改动，原来1e-4
# #                        'add_on_layers': 3e-3,
#                       'prototype_vectors':3e-3}
#   joint_optimizer_specs = \
#   [
#       {'params': meta_multi.features.parameters(), 'lr': joint_optimizer_lrs['features']}, # bias are now also being regularized
#     # {'params': meta_multi.conv2.parameters(), 'lr': joint_optimizer_lrs['features']},
#     # {'params': meta_multi.conv3.parameters(), 'lr': joint_optimizer_lrs['features']},
#     # {'params': meta_multi.conv4.parameters(), 'lr': joint_optimizer_lrs['features']},
#   #  {'params': meta_multi.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers']},
#     {'params': meta_multi.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
#   # {'params': meta_multi.logits.parameters(), 'lr':1e-4}
#   ]
#   joint_optimizer = torch.optim.Adam(joint_optimizer_specs)
#   loss_fn = nn.CrossEntropyLoss().to(device)
#   return meta_multi,joint_optimizer,loss_fn

In [22]:
#helper.py
def makedir(path):
  '''
  if path does not exist in the file system, create it
  '''
  if not os.path.exists(path):
    os.makedirs(path)



def find_high_activation_crop(activation_map, percentile=60):
    threshold = np.percentile(activation_map, percentile)
    mask = np.ones(activation_map.shape)
    mask[activation_map < threshold] = 0
    lower_y, upper_y, lower_x, upper_x = 0, 0, 0, 0
    for i in range(mask.shape[0]):
        if np.amax(mask[i]) > 0.5:
            lower_y = i
            break
    for i in reversed(range(mask.shape[0])):
        if np.amax(mask[i]) > 0.5:
            upper_y = i
            break
    for j in range(mask.shape[1]):
        if np.amax(mask[:,j]) > 0.5:
            lower_x = j
            break
    for j in reversed(range(mask.shape[1])):
        if np.amax(mask[:,j]) > 0.5:
            upper_x = j
            break
    return lower_y, upper_y+1, lower_x, upper_x+1

def coumpute_zjym_prototype(size_ori,size_now,position):
    patch=size_ori // size_now
    fmap_height_start_index = position[1] * patch
    fmap_height_end_index = (position[1]+1)*patch
    fmap_width_start_index = position[2] * patch
    fmap_width_end_index = (position[2]+1)*patch    
    return [position[0],fmap_height_start_index,fmap_height_end_index,fmap_width_start_index,fmap_width_end_index]

In [23]:
# def SimilarityConsis(model,
#         search_batch_input,
#         prototype_network_parallel,
#         fast_weights,
#         n_way,q_query,
#         search_y=None, # required if class_specific == True
#         num_classes=None):

#   search_batch = search_batch_input
#   with torch.no_grad():
#     search_batch = search_batch.cuda()
#     # this computation currently is not parallelized
#     _, proto_dist_torch = model.push_forward(search_batch,fast_weights)

#   proto_dist_ = np.copy(proto_dist_torch.detach().cpu().numpy())

#   del proto_dist_torch

#   class_to_img_index_dict = {key: [] for key in range(n_way)}
#       # img_y is the image's integer label
#   for img_index, img_y in enumerate(search_y):
#     img_label = img_y.item()
#     class_to_img_index_dict[img_label].append(img_index)

#   prototype_shape = model.prototype_shape
#   n_prototypes = prototype_shape[0]
#   x= list(np.zeros((q_query,n_prototypes)))
#   for j in range(n_prototypes):
#     target_class = torch.argmax(model.prototype_class_identity[j]).item()
#     proto_dist_j = proto_dist_[class_to_img_index_dict[target_class]][:,j,:,:]# class_to_img_index_dict[target_class]为原型所属的类，[:,j,:,:]为第j个原型。意思是这个类下的图片和第j个原型的结果。
#     for m in range(len(proto_dist_j)):
#       quer_proto_dist_j = proto_dist_j[m]
#       index=np.argmin(quer_proto_dist_j, axis=None)
#       x[m][j]=int(index)
#   res = 0
#   for i in range(len(x)):
#     if res ==0:
#       res=[[]]
#     else:
#       res.append([[]])
#     for j in range(len(x[0])):
#       res[i]=res[i]+[int(x[i][j])]
#   return res

In [24]:
# def update_prototypes_on_batch(search_batch_input,
#                 start_index_of_search_batch,
#                 prototype_network_parallel,
#                 global_min_proto_dist, # this will be updated
#                 global_min_fmap_patches, # this will be updated
#                 proto_rf_boxes, # this will be updated
#                 proto_bound_boxes, # this will be updated
#                 fast_weights,
#                 class_specific=True,
#                 search_y=None, # required if class_specific == True
#                 num_classes=None, # required if class_specific == True
#                 prototype_layer_stride=1,
#                 dir_for_saving_prototypes=None,
#                 prototype_img_filename_prefix=None,
#                 prototype_self_act_filename_prefix=None,
#                 prototype_activation_function_in_numpy=None):

# #     prototype_network_parallel.eval()


#       # print('preprocessing input for pushing ...')
#       # search_batch = copy.deepcopy(search_batch_input)
#   # 可能有问题
# #     search_batch = preprocess(search_batch_input)
#   search_batch = search_batch_input
#   with torch.no_grad():
#     search_batch = search_batch.cuda()
#     # this computation currently is not parallelized
#     protoL_input_torch, proto_dist_torch = prototype_network_parallel.push_forward(search_batch,fast_weights)

#   protoL_input_ = np.copy(protoL_input_torch.detach().cpu().numpy())
#   proto_dist_ = np.copy(proto_dist_torch.detach().cpu().numpy())

#   del protoL_input_torch, proto_dist_torch

#   if class_specific:
#     class_to_img_index_dict = {key: [] for key in range(num_classes)}
#     # img_y is the image's integer label
#     for img_index, img_y in enumerate(search_y):
#       img_label = img_y.item()
#       class_to_img_index_dict[img_label].append(img_index)

#   prototype_shape = prototype_network_parallel.prototype_shape
#   n_prototypes = prototype_shape[0]
#   proto_h = prototype_shape[2]
#   proto_w = prototype_shape[3]
#   max_dist = prototype_shape[1] * prototype_shape[2] * prototype_shape[3]

#   for j in range(n_prototypes):
#     #if n_prototypes_per_class != None:
#     if class_specific:
#         # target_class is the class of the class_specific prototype
#       target_class = torch.argmax(prototype_network_parallel.prototype_class_identity[j]).item()
#       # if there is not images of the target_class from this batch
#       # we go on to the next prototype
#       if len(class_to_img_index_dict[target_class]) == 0:
#         continue
#       proto_dist_j = proto_dist_[class_to_img_index_dict[target_class]][:,j,:,:]# class_to_img_index_dict[target_class]为原型所属的类，[:,j,:,:]为第j个原型。意思是这个类下的图片和第j个原型的结果。
#     else:
#       # if it is not class specific, then we will search through
#       # every example
#       proto_dist_j = proto_dist_[:,j,:,:]

#     batch_min_proto_dist_j = np.amin(proto_dist_j)# 第j个原型所属的第x类的最近的值。（激活最大的地方）
#     if batch_min_proto_dist_j < global_min_proto_dist[j]:# 
#       batch_argmin_proto_dist_j = \
#           list(np.unravel_index(np.argmin(proto_dist_j, axis=None),
#                                 proto_dist_j.shape))# batch最小原型距离的坐标。
#       if class_specific:
#         '''
#         change the argmin index from the index among
#         images of the target class to the index in the entire search
#         batch
#         '''
#         # 因为都是五分类，这一步可以省略。
#         batch_argmin_proto_dist_j[0] = class_to_img_index_dict[target_class][batch_argmin_proto_dist_j[0]]

#       # retrieve the corresponding feature map patch
#       img_index_in_batch = batch_argmin_proto_dist_j[0]
#       fmap_height_start_index = batch_argmin_proto_dist_j[1] * prototype_layer_stride
#       fmap_height_end_index = fmap_height_start_index + proto_h
#       fmap_width_start_index = batch_argmin_proto_dist_j[2] * prototype_layer_stride
#       fmap_width_end_index = fmap_width_start_index + proto_w
#       # 对应的features(x),对应的位置。
#       batch_min_fmap_patch_j = protoL_input_[img_index_in_batch,
#                                               :,
#                                               fmap_height_start_index:fmap_height_end_index,
#                                               fmap_width_start_index:fmap_width_end_index]
#       # 此处应该会修改，毕竟每个五分类任务的训练集只有一个。
#       global_min_proto_dist[j] = batch_min_proto_dist_j
#       global_min_fmap_patches[j] = batch_min_fmap_patch_j
      
#       # get the receptive field boundary of the image patch
#       # that generates the representation
# #             protoL_rf_info = prototype_network_parallel.proto_layer_rf_info
# #             rf_prototype_j = compute_rf_prototype(search_batch.size(2), batch_argmin_proto_dist_j, protoL_rf_info)
#       rf_prototype_j = coumpute_zjym_prototype(search_batch.size(2),protoL_input_.shape[2],batch_argmin_proto_dist_j)
#       # get the whole image
#       original_img_j = search_batch_input[rf_prototype_j[0]]
#       original_img_j = original_img_j.data.cpu().numpy()
#       original_img_j = np.transpose(original_img_j, (1, 2, 0))
#       original_img_size = original_img_j.shape[0]#28,28,1
      
#       # crop out the receptive field
#       rf_img_j = original_img_j[rf_prototype_j[1]:rf_prototype_j[2],
#                                 rf_prototype_j[3]:rf_prototype_j[4], :]
      
#       # save the prototype receptive field information
#       proto_rf_boxes[j, 0] = rf_prototype_j[0] + start_index_of_search_batch
#       proto_rf_boxes[j, 1] = rf_prototype_j[1]
#       proto_rf_boxes[j, 2] = rf_prototype_j[2]
#       proto_rf_boxes[j, 3] = rf_prototype_j[3]
#       proto_rf_boxes[j, 4] = rf_prototype_j[4]
#       if proto_rf_boxes.shape[1] == 6 and search_y is not None:# 记录类别。
#           proto_rf_boxes[j, 5] = search_y[rf_prototype_j[0]].item()

#       # find the highly activated region of the original image
#       proto_dist_img_j = proto_dist_[img_index_in_batch, j, :, :]
#       if prototype_network_parallel.prototype_activation_function == 'log':
#           proto_act_img_j = np.log((proto_dist_img_j + 1) / (proto_dist_img_j + prototype_network_parallel.epsilon))
#       elif prototype_network_parallel.prototype_activation_function == 'linear':
#           proto_act_img_j = max_dist - proto_dist_img_j
#       else:
#           proto_act_img_j = prototype_activation_function_in_numpy(proto_dist_img_j)
#       upsampled_act_img_j = cv2.resize(proto_act_img_j, dsize=(original_img_size, original_img_size),
#                                         interpolation=cv2.INTER_CUBIC)
#       proto_bound_j = find_high_activation_crop(upsampled_act_img_j)
#       # crop out the image patch with high activation as prototype image
#       proto_img_j = original_img_j[proto_bound_j[0]:proto_bound_j[1],
#                                     proto_bound_j[2]:proto_bound_j[3], :]

#       # save the prototype boundary (rectangular boundary of highly activated region)
#       proto_bound_boxes[j, 0] = proto_rf_boxes[j, 0]
#       proto_bound_boxes[j, 1] = proto_bound_j[0]
#       proto_bound_boxes[j, 2] = proto_bound_j[1]
#       proto_bound_boxes[j, 3] = proto_bound_j[2]
#       proto_bound_boxes[j, 4] = proto_bound_j[3]
#       if proto_bound_boxes.shape[1] == 6 and search_y is not None:
#         proto_bound_boxes[j, 5] = search_y[rf_prototype_j[0]].item()

#       if dir_for_saving_prototypes is not None:
#         if prototype_self_act_filename_prefix is not None:
#           # save the numpy array of the prototype self activation
#           np.save(os.path.join(dir_for_saving_prototypes,
#                     prototype_self_act_filename_prefix + str(j) + '.npy'),
#                     proto_act_img_j)
#         if prototype_img_filename_prefix is not None:
#             # save the whole image containing the prototype as png
# #                     plusBB(original_img_j,proto_bound_j[0],proto_bound_j[1],proto_bound_j[2],proto_bound_j[3],os.path.join(dir_for_saving_prototypes,
# #                                             prototype_img_filename_prefix + '-original' + str(j) + '.png'))
# #                     cv2.rectangle(original_img_j,(int(proto_bound_j[0]),int(proto_bound_j[1])),(int(proto_bound_j[2]),int(proto_bound_j[3])),(0,255,0),3)  
#           plt.imsave(os.path.join(dir_for_saving_prototypes,
#                                   prototype_img_filename_prefix + '-original' + str(j) + '.png'),
#                       original_img_j,
# #                                np.squeeze(original_img_j),
#                       vmin=0.0,
#                       vmax=1.0)
#           # overlay (upsampled) self activation on original image and save the result
#           rescaled_act_img_j = upsampled_act_img_j - np.amin(upsampled_act_img_j)
#           rescaled_act_img_j = rescaled_act_img_j / np.amax(rescaled_act_img_j)
#           heatmap = cv2.applyColorMap(np.uint8(255*rescaled_act_img_j), cv2.COLORMAP_JET)
#           heatmap = np.float32(heatmap) / 255
#           heatmap = heatmap[...,::-1]
#           overlayed_original_img_j = 0.5 * original_img_j + 0.3 * heatmap
#           plt.imsave(os.path.join(dir_for_saving_prototypes,
#                                   prototype_img_filename_prefix + '-original_with_self_act' + str(j) + '.png'),
#                       overlayed_original_img_j,
#                       vmin=0.0,
#                       vmax=1.0)
          
#           # if different from the original (whole) image, save the prototype receptive field as png
#           if rf_img_j.shape[0] != original_img_size or rf_img_j.shape[1] != original_img_size:
#             plt.imsave(os.path.join(dir_for_saving_prototypes,
#                                     prototype_img_filename_prefix + '-receptive_field' + str(j) + '.png'),
# #                                    np.squeeze(rf_img_j),
#                         rf_img_j,
#                         vmin=0.0,
#                         vmax=1.0)
#             overlayed_rf_img_j = overlayed_original_img_j[rf_prototype_j[1]:rf_prototype_j[2],
#                                                           rf_prototype_j[3]:rf_prototype_j[4]]
#             plt.imsave(os.path.join(dir_for_saving_prototypes,
#                                     prototype_img_filename_prefix + '-receptive_field_with_self_act' + str(j) + '.png'),
#                         overlayed_rf_img_j,
#                         vmin=0.0,
#                         vmax=1.0)
          
#           # save the prototype image (highly activated region of the whole image)
#           plt.imsave(os.path.join(dir_for_saving_prototypes,
#                 prototype_img_filename_prefix + str(j) + '.png'),
# #                                 np.squeeze(proto_img_j),
#                   proto_img_j,
#                   vmin=0.0,
#                   vmax=1.0)
          
#   if class_specific:
#       del class_to_img_index_dict


In [25]:


def toList(tensor):
    res = np.copy(tensor.detach().cpu().numpy())
    return res.tolist()


In [26]:
def get_meta_batch(meta_batch_size,
                   k_shot, q_query, 
                   data_loader, iterator):
    data = []
    label_lst =[]
    for _ in range(meta_batch_size):
        try:
            # a "task_data" tensor is representing \
            #     the data of a task, with size of \
            #     [n_way, k_shot+q_query, 1, 28, 28]
            task_data = iterator.next()  
        except StopIteration:
            iterator = iter(data_loader)
            task_data = iterator.next()
        label = task_data[0]
        label_lst.append(label)
        task_data = task_data[1]
        train_data = (task_data[:, :k_shot]
                      .reshape(-1, 3, 224, 224))
        val_data = (task_data[:, k_shot:]
                    .reshape(-1, 3,224, 224))
        task_data = torch.cat(
            (train_data, val_data), 0)
        data.append(task_data)
    return torch.stack(data).to(device), iterator,label_lst

In [27]:
def labeltoindex(label):
    with open('/content/imagenet_index_class.json', 'r', encoding='utf8')as fp:
        labels_all = json.load(fp)
    index = labels_all[label]
    return index

In [28]:
def find_label(label_lst,k_shot):
    labels = list(label_lst)
    index_lst=[]
    for label in labels:
        index = labeltoindex(label)
        index_lst=index_lst+[index]*k_shot
    # index_lst.append(labeltoindex(label))
    return torch.tensor(index_lst)

In [29]:
def MetaAlgorithmGenerator(
    inner_update      = inner_update_MAML, 
    collect_gradients = collect_gradients_MAML, 
    outer_update      = outer_update_MAML):

    global calculate_accuracy

    def MetaAlgorithm(
        model, optimizer, x, n_way, k_shot, q_query, loss_fn,label_lst,
        inner_train_step=5, inner_lr=0.01, train=True): 
        criterion = loss_fn
        task_loss, task_acc,task_pre,task_recall,task_f1 = [], [],[],[],[]
        special_grad = OrderedDict()  # Added for variants!
#         coefs = {
#             'crs_ent': 1,
#             'clst': 0.8,
#             'sep': -0.08,
#             'l1': 1e-4,
#             }
        for task_index,meta_batch in enumerate(x):
            support_set = meta_batch[: n_way * k_shot]  
            query_set = meta_batch[n_way * k_shot :]    

            fast_weights = OrderedDict(model.named_parameters())

          ### ---------- INNER TRAIN LOOP ---------- ###
            for inner_step in range(inner_train_step): 
                train_label = create_label(n_way, k_shot).to(device)
                logits = model.functional_forward(support_set, fast_weights)
                loss = criterion(logits, train_label)
                # logits = predict_conv(support_set)
                # loss = criterion(logits, train_label)
                fast_weights = inner_update(fast_weights, loss, inner_lr)

          ### ---------- INNER VALID LOOP ---------- ###
          # val_label = create_label(label_lst[task_index], q_query).to(device)
            val_label=create_label(n_way, k_shot).to(device)
          # FIXME: W for val?
          # special_grad = collect_gradients(
          #     special_grad, fast_weights, model, len(x))

          # Collect gradients for outer loop
            logits = model.functional_forward(query_set, fast_weights)
          # logits = model.functional_forward(query_set, fast_weights) 
            loss = criterion(logits, val_label)
            task_loss.append(loss)
            acc,precision,recall,f1 = calculate_accuracy(logits, val_label,n_way,q_query)
            task_acc.append(acc)
            task_pre.append(precision)
            task_recall.append(recall)
            task_f1.append(f1)
        # Update outer loop
        # model.train()
        # optimizer.zero_grad()

        meta_batch_loss = torch.stack(task_loss).mean()
        if train:
            # Notice the update part!
            outer_update(model, meta_batch_loss, special_grad)
            optimizer.step()
        task_acc = np.mean(task_acc)
        task_pre=np.mean(task_pre)
        task_recall=np.mean(task_recall)
        task_f1 = np.mean(task_f1)
        return meta_batch_loss, task_acc,task_pre,task_recall,task_f1
    return MetaAlgorithm

In [30]:
# def MetaAlgorithmGeneratorNew(
#   inner_update      = inner_update_MAML, 
#   collect_gradients = collect_gradients_MAML, 
#   outer_update      = outer_update_MAML):

#   global calculate_accuracy

#   def MetaAlgorithmNew(
#     model, optimizer, x, n_way, k_shot, q_query, loss_fn,epoch_number,step,
#     inner_train_step=10, inner_lr=0.04, train=0): 
#     criterion = loss_fn
#     task_loss, task_acc = [], []
#     special_grad = OrderedDict()  # Added for variants!
#     coefs = {
#         'crs_ent': 1,
#         'clst': 0.8,
#         'sep': -0.8,
#         'l1': 1e-4,
#         }
#     task_crsEnt,task_cls,task_sep,task_l1 = [],[],[],[]
#     task_dic={}
#     task_dic_path='./test'
#     makedir(task_dic_path)
#     for task_index,meta_batch in enumerate(x):
# #             print(task_index)
#       support_set = meta_batch[: n_way * k_shot]  
#       query_set = meta_batch[n_way * k_shot :]    
      
#       fast_weights = OrderedDict(model.named_parameters())
      
#       ### ---------- INNER TRAIN LOOP ---------- ###
#       for inner_step in range(inner_train_step): 
#         train_label = create_label(n_way, k_shot).to(device)
#         logits,min_distances = model.functional_forward(support_set, fast_weights)
#         cross_entropy = criterion(logits, train_label)

#         max_dist = (model.prototype_shape[1]
#                     * model.prototype_shape[2]
#                     * model.prototype_shape[3])
#         # calculate cluster cost                
#         prototypes_of_correct_class = torch.t(model.prototype_class_identity[:,train_label]).cuda()
#         inverted_distances, _ = torch.max((max_dist - min_distances) * prototypes_of_correct_class, dim=1)
#         cluster_cost = torch.mean(max_dist - inverted_distances)
        
        
#         # calculate separation cost
#         prototypes_of_wrong_class = 1 - prototypes_of_correct_class
#         inverted_distances_to_nontarget_prototypes, _ = \
#             torch.max((max_dist - min_distances) * prototypes_of_wrong_class, dim=1)
#         separation_cost = torch.mean(max_dist - inverted_distances_to_nontarget_prototypes)
            
#         # calculate avg cluster cost
#         avg_separation_cost = \
#             torch.sum(min_distances * prototypes_of_wrong_class, dim=1) / torch.sum(prototypes_of_wrong_class, dim=1)
#         avg_separation_cost = torch.mean(avg_separation_cost)
        
        
#         l1_mask = 1 - torch.t(model.prototype_class_identity).cuda()
#         l1 = (model.logits.weight * l1_mask).norm(p=1)
#         loss = (coefs['crs_ent'] * cross_entropy
#                           + coefs['clst'] * cluster_cost
#                           + coefs['sep'] * separation_cost
#                           + coefs['l1'] * l1)
#         ones =  fast_weights.pop('ones')
#         logitstemp = fast_weights.pop('logits.weight')
#         fast_weights = inner_update(fast_weights, loss, inner_lr)
        
#         fast_weights['ones'] = ones
#         fast_weights['logits.weight'] = logitstemp
      
#       # push prototype
#       if epoch_number>27:
#         train_label = create_label(n_way,k_shot).to(device)

#         prototype_shape = model.prototype_shape
#         n_prototypes = model.num_prototypes
#         global_min_proto_dist = np.full(n_prototypes, np.inf)
#         global_min_fmap_patches = np.zeros(
#             [n_prototypes,
#               prototype_shape[1],
#               prototype_shape[2],
#               prototype_shape[3]])
#         proto_rf_boxes = np.full(shape=[n_prototypes, 6],
#                             fill_value=-1)
#         proto_bound_boxes = np.full(shape=[n_prototypes, 6],
#                                     fill_value=-1)
#         makedir('./img')
#         root_dir_for_saving_prototypes = os.path.join('./img', 'epoch-'+str(epoch_number))
#         task_dic_path = os.path.join(task_dic_path,'epoch-'+str(epoch_number))
#         if train==0:
#           root_dir_for_saving_prototypes = os.path.join(root_dir_for_saving_prototypes, 'train')
#         elif train==1:
#           root_dir_for_saving_prototypes = os.path.join(root_dir_for_saving_prototypes, 'val')
#         else:
#           root_dir_for_saving_prototypes = os.path.join('./test','epoch-'+str(epoch_number))
#           root_dir_for_saving_prototypes =os.path.join(root_dir_for_saving_prototypes,'prototype')
#         root_dir_for_saving_prototypes = os.path.join(root_dir_for_saving_prototypes,'step-'+str(step))
        
#         makedir(root_dir_for_saving_prototypes)
#         # 此处的epoch_number可能要传epoch进来
# #                 print(os.path.join(root_dir_for_saving_prototypes,'task-'+str(task_index)))
#         proto_epoch_dir =os.path.join(root_dir_for_saving_prototypes,'task-'+str(task_index))
        
#         makedir(proto_epoch_dir)
#         search_batch_size =n_way

#         num_classes = model.num_classes
# #             for push_iter, search_batch_inpu in eupdate_prototypes_on_batchnumerate(support_set):

#         start_index_of_search_batch = 0

#         update_prototypes_on_batch(support_set,
#                             start_index_of_search_batch,
#                             model,
#                             global_min_proto_dist,
#                             global_min_fmap_patches,
#                             proto_rf_boxes,
#                             proto_bound_boxes,
#                             fast_weights,
#                             class_specific=True,
#                             search_y=train_label,
#                             num_classes=num_classes,
#                             prototype_layer_stride=1,
#                             dir_for_saving_prototypes=proto_epoch_dir,
#                             prototype_img_filename_prefix='prototype-img',
#                             prototype_self_act_filename_prefix='prototype-self-act')    

#         proto_bound_boxes_filename_prefix='bb'
#         # proto_bound_boxes_filename_prefix ='bb'=='None'
#         if proto_epoch_dir != None and proto_bound_boxes_filename_prefix != None:
#           np.save(os.path.join(proto_epoch_dir, proto_bound_boxes_filename_prefix + str(epoch_number) + '.npy'),
#                     proto_bound_boxes)     

#         prototype_update = np.reshape(global_min_fmap_patches,
#                               tuple(prototype_shape))
        
#         fast_weights['prototype_vectors'] = torch.tensor(prototype_update, dtype=torch.float32).cuda()
      
      
      
#       ### ---------- INNER VALID LOOP ---------- ###
#       # ----inner test-----#
#       val_label = create_label(n_way, q_query).to(device)
#       # FIXME: W for val?
#       ones =  fast_weights.pop('ones')            
#       logitstemp = fast_weights.pop('logits.weight')
      
#       special_grad = collect_gradients(
#           special_grad, fast_weights, model, len(x))
      
#       fast_weights['ones'] = ones
#       fast_weights['logits.weight'] = logitstemp
#       # Collect gradients for outer loop
#       logits,min_distances = model.functional_forward(query_set, fast_weights) 
      
#       cross_entropy = criterion(logits, val_label)
#       all_dic={}
#       index_dic={}
#       high_dic={}
#       if train ==2:
#         if epoch_number>27:
#           task_dic_path = os.path.join('./test','epoch-'+str(epoch_number))
#           task_dic_path = os.path.join(task_dic_path,'original')
#           task_dic_path = os.path.join(task_dic_path,'step-'+str(step))
#           task_dic_path = os.path.join(task_dic_path,'task-'+str(task_index))
#           makedir(task_dic_path)
# #                 pp_distance=min_distances.cpu().numpy()
#           for t in range(len(val_label)):
#             original_img_j = query_set[t]
#             original_img_j = original_img_j.data.cpu().numpy()
#             original_img_j = np.transpose(original_img_j, (1, 2, 0))
#             original_img_size = original_img_j.shape[0]
#             if t % q_query ==0:
#                 final_path = os.path.join(task_dic_path,'class-'+str(t//q_query))
#                 makedir(final_path)
                
#             plt.imsave(os.path.join(final_path,str(t%q_query)+'.png'),
#                     original_img_j,
#                     vmin=0.0,
#                     vmax=1.0)
              
#           task_dic['task-'+str(task_index)]=toList(min_distances)
#           all_dic['res']=predict_true(logits,val_label)
#           high_index= SimilarityConsis(model,query_set,model,fast_weights,n_way,q_query,search_y=val_label,num_classes=num_classes)
#           index_dic['index']=high_index
#           high_score =  torch.log((min_distances + 1) / (min_distances + model.epsilon))
#           high_dic['res']=toList(high_score)
#           jsondata = json.dumps(task_dic,indent=4)
#           jsondata2 = json.dumps(all_dic,indent=4)
#           jsondata3 = json.dumps(index_dic,indent=4)
#           jsondata4 = json.dumps(high_dic,indent=4)
#           f = open(task_dic_path+'/distance.json', 'w')
#           f2= open(task_dic_path+'/predict_res.json','w')
#           f3 =open(task_dic_path+'/index.json','w')
#           f4=open(task_dic_path+'/activation.json','w')
#           f.write(jsondata)
#           f2.write(jsondata2)
#           f3.write(jsondata3)
#           f4.write(jsondata4)
#           f.close()
#           f2.close()
#           f3.close()
#           f4.close()
#       max_dist = (model.prototype_shape[1]
#           * model.prototype_shape[2]
#           * model.prototype_shape[3])
      
#       # calculate cluster cost                
#       prototypes_of_correct_class = torch.t(model.prototype_class_identity[:,val_label]).cuda()
#       inverted_distances, _ = torch.max((max_dist - min_distances) * prototypes_of_correct_class, dim=1)
#       cluster_cost = torch.mean(max_dist - inverted_distances)


#       # calculate separation cost
#       prototypes_of_wrong_class = 1 - prototypes_of_correct_class
#       inverted_distances_to_nontarget_prototypes, _ = \
#           torch.max((max_dist - min_distances) * prototypes_of_wrong_class, dim=1)
#       separation_cost = torch.mean(max_dist - inverted_distances_to_nontarget_prototypes)

#       # calculate avg cluster cost
#       avg_separation_cost = \
#           torch.sum(min_distances * prototypes_of_wrong_class, dim=1) / torch.sum(prototypes_of_wrong_class, dim=1)
#       avg_separation_cost = torch.mean(avg_separation_cost)
#       l1_mask = 1 - torch.t(model.prototype_class_identity).cuda()
#       l1 = (model.logits.weight * l1_mask).norm(p=1)
      
#       loss = (coefs['crs_ent'] * cross_entropy
#                         + coefs['clst'] * cluster_cost
#                         + coefs['sep'] * separation_cost
#                         + coefs['l1'] * l1)
      
#       task_loss.append(loss)
#       task_acc.append(calculate_accuracy(logits, val_label))
#       task_crsEnt.append(cross_entropy)
#       task_cls.append(cluster_cost)
#       task_sep.append(separation_cost)
#       task_l1.append(l1)
# #             task_acc.append(calculate_accuracy(logits, val_label))
#     # Update outer loop
#     model.train()
#     optimizer.zero_grad()

#     meta_batch_loss = torch.stack(task_loss).mean()
#     meta_batch_crsEnt=torch.stack(task_crsEnt).mean()
#     meta_batch_cls =torch.stack(task_cls).mean()
#     meta_batch_sep = torch.stack(task_sep).mean()
#     meta_batch_l1 = torch.stack(task_l1).mean()
#     if train==0:
#       ones =  fast_weights.pop('ones')
#       logitstemp = fast_weights.pop('logits.weight')
#       # Notice the update part!
#       outer_update(model, meta_batch_loss, special_grad)
#       fast_weights['ones'] = ones
#       fast_weights['logits.weight'] = logitstemp            
#       optimizer.step()
#     task_acc = np.mean(task_acc)
#     return meta_batch_loss, task_acc,meta_batch_crsEnt,meta_batch_cls,meta_batch_sep,meta_batch_l1
#   return MetaAlgorithmNew

In [31]:
def ori_outer_train_val(max_epoch
            ,meta_model
            ,MetaAlgorithm
            ,optimizer
            ,loss_fn
            ,train_loader,val_loader
            ,train_iter,val_iter
            ,meta_batch_size,eval_batches
            ,n_way
            ,k_shot
            ,q_query
            ,inner_train_step=10
            ,train=False):
    for epoch in range(max_epoch):
        print("Epoch %d" % (epoch + 1))
        train_meta_loss = []
        train_acc = []
        train_pre,train_recall,train_f1=[],[],[]
        # The "step" here is a meta-gradinet update step
        for step in tqdm(range(5)):
                # len(train_loader) // meta_batch_size)): 
            x, train_iter,label_lst = get_meta_batch(
              meta_batch_size, k_shot, q_query, 
              train_loader, train_iter)
            meta_loss, acc,precision,recall,f1= MetaAlgorithm(
              meta_model, optimizer, x, 
              n_way, k_shot, q_query, loss_fn,label_lst)
            train_meta_loss.append(meta_loss.item())
            train_acc.append(acc)
            train_pre.append(precision)
            train_recall.append(recall)
            train_f1.append(f1)
        print("  Loss    : ", "%.3f" % (np.mean(train_meta_loss)), end='\t')
        print("  Accuracy: ", "%.3f %%" % (np.mean(train_acc) * 100))
        print("  Precision: ", "%.3f %%" % (np.mean(train_pre) * 100),end='\t')
        print("  Recall: ", "%.3f %%" % (np.mean(train_recall) * 100),end='\t')
        print("  F1: ", "%.3f %%" % (np.mean(train_f1) * 100),end='\t')

        # See the validation accuracy after each epoch.
        # Early stopping is welcomed to implement.
        val_acc = []
        val_pre,val_recall,val_f1=[],[],[]
        for eval_step in tqdm(range(6)):
                # len(val_loader) // (eval_batches))):
            x, val_iter,label_lst = get_meta_batch(
              eval_batches, k_shot, q_query, 
              val_loader, val_iter)
          # We update three inner steps when testing.
            _, acc,precision,recall,f1 = MetaAlgorithm(meta_model, optimizer, x, 
                        n_way, k_shot, q_query, 
                        loss_fn, label_lst,
                        inner_train_step=inner_train_step, 
                        train=train) 
            val_acc.append(acc)
            val_pre.append(precision)
            val_recall.append(recall)
            val_f1.append(f1)
        print("  Validation accuracy: ", "%.3f %%" % (np.mean(val_acc) * 100))
        print("  Validation precision: ", "%.3f %%" % (np.mean(val_pre) * 100))
        print("  Validation recall: ", "%.3f %%" % (np.mean(val_recall) * 100))
        print("  Validation f1: ", "%.3f %%" % (np.mean(val_f1) * 100))
    #         if np.mean(val_acc)>0.985:
    #             break
    return meta_model

In [32]:
def ori_outer_test(meta_model,MetaAlgorithm,optimizer,loss_fn,test_loader,test_iter,test_batches,n_way,k_shot,q_query,inner_train_step=10,train=False):
    test_acc = []
    test_pre,test_recall,test_f1=[],[],[]
    for test_step in tqdm(range(6)):
          # len(test_loader) // (test_batches))):
        x, test_iter,label_lst = get_meta_batch(
            test_batches, k_shot, q_query, 
            test_loader, test_iter)
    # When testing, we update 3 inner-steps
        _, acc,precision,recall,f1 = MetaAlgorithm(meta_model, optimizer, x, 
                  n_way, k_shot, q_query, loss_fn, label_lst,
                  inner_train_step=inner_train_step, train=False)
        test_acc.append(acc)
        test_pre.append(precision)
        test_recall.append(recall)
        test_f1.append(f1)
    print("  Testing accuracy: ", "%.3f %%" % (np.mean(test_acc) * 100))
    print("  Testing precision: ", "%.3f %%" % (np.mean(test_pre) * 100))
    print("  Testing recall: ", "%.3f %%" % (np.mean(test_recall) * 100))
    print("  Testing f1: ", "%.3f %%" % (np.mean(test_f1) * 100))
    return meta_model

In [33]:
def ori_outer(n_way,k_shot,q_query,in_ch,meta_lr,max_epoch,train_loader,val_loader,test_loader,train_iter,val_iter,test_iter
            ,meta_batch_size,eval_batches,test_batches,inner_train_step=10,train=False,model_ori_name=False,optimizer=False):
    if optimizer != False:
        print("1111")
        meta_model,_,loss_fn=model_init(n_way,in_ch,meta_lr)
    else:
        meta_model, optimizer, loss_fn = model_init(n_way,in_ch,meta_lr)
    MetaAlgorithm = MetaAlgorithmGenerator()
    try:
        meta_model = torch.load(model_ori_name)
    except:
        print("can't find original model's name, please check 'model_ori_name'")
    meta_model=ori_outer_train_val(max_epoch
                  ,meta_model
                  ,MetaAlgorithm
                  ,optimizer
                  ,loss_fn
                  ,train_loader,val_loader
                  ,train_iter,val_iter
                  ,meta_batch_size,eval_batches
                  ,n_way
                  ,k_shot
                  ,q_query
                  ,inner_train_step=inner_train_step
                  ,train=train)
    meta_model=ori_outer_test(meta_model
                ,MetaAlgorithm
                ,optimizer
                ,loss_fn
                ,test_loader
                ,test_iter
                ,test_batches
                ,n_way
                ,k_shot
                ,q_query
                ,inner_train_step=inner_train_step
                ,train=train)
  
    return meta_model,optimizer

In [34]:
# def new_outer_train_val(max_epoch
#               ,meta_model
#               ,MetaAlgorithm_new
#               ,optimizer
#               ,loss_fn
#               ,train_loader,val_loader
#               ,train_iter,val_iter
#               ,meta_batch_size,eval_batches
#               ,n_way
#               ,k_shot
#               ,q_query
#               ,inner_train_step=15
#               ,train=1):
#     for epoch in range(max_epoch):
#         print("Epoch %d" % (epoch + 1))
#         train_meta_loss = []
#         train_acc = []
#         train_meta_crsEnt,train_meta_cls,train_meta_sep,train_meta_l1=[],[],[],[]
#     # The "step" here is a meta-gradinet update step
#         for step in tqdm(range(5)):
#             # len(train_loader) // meta_batch_size)): 
#             x, train_iter,label_lst = get_meta_batch(
#                 meta_batch_size, k_shot, q_query, 
#                 train_loader, train_iter)
#             meta_loss, acc= MetaAlgorithm(
#               meta_model, optimizer, x, 
#               n_way, k_shot, q_query, loss_fn,label_lst,epoch,step)
#             train_meta_loss.append(meta_loss.item())
#             train_acc.append(acc)
 

#     print("  Loss    : ", "%.3f" % (np.mean(train_meta_loss)), end='\t')
#     print("  Accuracy: ", "%.3f %%" % (np.mean(train_acc) * 100),end='\t')
#     # print("  crsEnt    : ", "%.3f" % (np.mean(train_meta_crsEnt)), end='\t')
#     # print("  cls    : ", "%.3f" % (np.mean(train_meta_cls)), end='\t')
#     # print("  sep    : ", "%.3f" % (np.mean(train_meta_sep)), end='\t')
#     # print("  l1    : ", "%.3f" % (np.mean(train_meta_l1)))

# #     See the validation accuracy after each epoch.
# #     Early stopping is welcomed to implement.
#     val_acc = []
#     for eval_step in tqdm(range(6)):
#             # len(val_loader) // (eval_batches))):
#       x, val_iter,label_lst = get_meta_batch(
#                 eval_batches, k_shot, q_query, 
#                 val_loader, val_iter)
#             # We update three inner steps when testing.
#       _, acc = MetaAlgorithm(meta_model, optimizer, x, 
#                     n_way, k_shot, q_query, 
#                     loss_fn,label_lst, 
#                     epoch,
#                     eval_step,
#                     inner_train_step=inner_train_step, 
#                     train=train
#                     )  
#       val_acc.append(acc)
#     print("  Validation accuracy: ", "%.3f %%" % (np.mean(val_acc) * 100))
#   return meta_model

In [35]:
# def new_outer_test(meta_model,MetaAlgorithm_new,optimizer,loss_fn,test_loader,test_iter,test_batches,n_way,k_shot,q_query,inner_train_step=15,train=2):
#   test_acc = []
#   for test_step in tqdm(range(6)):
#           # len(test_loader) // (test_batches))):
#     x, test_iter,label_lst = get_meta_batch(
#             test_batches, k_shot, q_query, 
#             test_loader, test_iter)
#     # When testing, we update 3 inner-steps
#     _, acc = MetaAlgorithm_new(meta_model, optimizer, x, 
#             n_way, k_shot, q_query, 
#                       loss_fn,
#                       30,
#                       test_step,
#                       inner_train_step=inner_train_step, 
#                       train=train) 
#     test_acc.append(acc)
#   print("  Testing accuracy: ", "%.3f %%" % (np.mean(test_acc) * 100))
#   return meta_model

In [36]:
# def copyConv(model_name,meta_model):
#   model_ori=torch.load(model_name)
#   meta_model.conv1.data.copy_(model_ori.conv1.data)
#   meta_model.conv2.data.copy_(model_ori.conv2.data)
#   meta_model.conv3.data.copy_(model_ori.conv3.data)
#   meta_model.conv4.data.copy_(model_ori.conv4.data)
#   return meta_model

In [37]:
# def new_outer(n_way,k_shot,q_query,in_ch,meta_lr,max_epoch,train_loader,val_loader,test_loader,train_iter,val_iter,test_iter
#             ,meta_batch_size,eval_batches,test_batches,prototype_shape,img_size,epsilon,prototype_activation_function,inner_train_step=15,train=False,model_ori_name=False):
#   meta_model, optimizer, loss_fn = model_init_new(n_way,in_ch,meta_lr,prototype_shape,img_size,epsilon,prototype_activation_function)
#   MetaAlgorithm_new = MetaAlgorithmGeneratorNew()    
#   try:
#     meta_model = copyConv(model_ori_name,meta_model)
#   except:
#     print("can't find original model's name, please check 'model_ori_name'")
#   meta_model=new_outer_train_val(max_epoch
#                 ,meta_model
#                 ,MetaAlgorithm_new
#                 ,optimizer
#                 ,loss_fn
#                 ,train_loader,val_loader
#                 ,train_iter,val_iter
#                 ,meta_batch_size,eval_batches
#                 ,n_way
#                 ,k_shot
#                 ,q_query
#                 ,inner_train_step=inner_train_step
#                 ,train=train)
#   meta_model=new_outer_test(meta_model
#               ,MetaAlgorithm_new
#               ,optimizer
#               ,loss_fn
#               ,test_loader
#               ,test_iter
#               ,test_batches
#               ,n_way
#               ,k_shot
#               ,q_query
#               ,inner_train_step=inner_train_step
#               ,train=train)
  
#   return meta_model

In [38]:
# def main():
n_way = 5
k_shot =1
q_query =1
# inner_train_step = 1
# inner_lr = 0.01
meta_lr = 0.001
meta_batch_size = 4
max_epoch = 5
eval_batches = test_batches = 4
in_ch=3
prototype_shape=(20,64,1,1)
img_size=224
epsilon=1e-4
prototype_activation_function='log'
train_data_path = '/home/featurize/data/train/'
val_data_path = '/home/featurize/data/val/'
test_data_path = '/home/featurize/data/test/'
# vgg16_bn_f = vgg16_bn_features(pretrained=True)
# print(vgg16_bn_f)
train_set = MiniImageNet(train_data_path, k_shot, q_query)
val_set = MiniImageNet(val_data_path,k_shot,q_query)
test_set = MiniImageNet(test_data_path, k_shot, q_query)

(train_loader, val_loader, test_loader), \
(train_iter, val_iter, test_iter) = dataloader_init(
                    (train_set, val_set, test_set),n_way)

#   
# meta_model,optimizer=ori_outer(n_way,k_shot,q_query,in_ch,meta_lr,max_epoch,train_loader,val_loader,test_loader,train_iter,val_iter,test_iter
#               ,meta_batch_size,eval_batches,test_batches,inner_train_step=10,train=False,model_ori_name=False,optimizer=False)
# return meta_model
  # meta_new = new_outer(n_way,k_shot,q_query,in_ch,meta_lr,max_epoch,train_loader,val_loader,test_loader,train_iter,val_iter,test_iter
  #          ,meta_batch_size,eval_batches,test_batches,prototype_shape,img_size,epsilon,prototype_activation_function,inner_train_step=15,train=False,model_ori_name='/content/drive/MyDrive/origin_20way1shot')
  # return meta_new

In [39]:
meta_model,optimizer=ori_outer(n_way,k_shot,q_query,in_ch,meta_lr,max_epoch,train_loader,val_loader,test_loader,train_iter,val_iter,test_iter
              ,meta_batch_size,eval_batches,test_batches,inner_train_step=5,train=False,model_ori_name=False,optimizer=False)

can't find original model's name, please check 'model_ori_name'
Epoch 1


 Iters    Elapsed Time      Speed                                               
  5/5   00:01:24<00:00:00  0.06it/s                                             

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Loss    :  3.613	  Accuracy:  54.000 %
  Precision:  40.583 %	  Recall:  54.000 %	  F1:  44.167 %	

 Iters    Elapsed Time      Speed                                               
  6/6   00:02:16<00:00:00  0.04it/s                                             

...

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Validation accuracy:  25.833 %
  Validation precision:  10.347 %
  Validation recall:  25.833 %
  Validation f1:  13.389 %


 Iters    Elapsed Time      Speed                                               
  6/6   00:01:31<00:00:00  0.07it/s                                             

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Testing accuracy:  25.833 %
  Testing precision:  10.361 %
  Testing recall:  25.833 %
  Testing f1:  13.611 %


In [40]:
# meta_model,_,loss_fn=model_init(n_way,in_ch,meta_lr)


In [41]:
torch.load('')

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
# print(meta_model.features)

In [ ]:
# x1=OrderedDict(meta_model.named_parameters())
# print(x1)

In [ ]:
# torch.save(meta_model,'best3')

In [ ]:
# torch.save(meta_model,'origin_20way1shot')

In [ ]:
meta_model, optimizer, loss_fn = model_init(n_way,in_ch,meta_lr)
MetaAlgorithm = MetaAlgorithmGenerator()


In [ ]:
x, train_iter = get_meta_batch(
    meta_batch_size, k_shot, q_query, 
    train_loader, train_iter)

In [ ]:
for meta_batch in x:
      # support_set = meta_batch[: n_way * k_shot]  
      # query_set = meta_batch[n_way * k_shot :]
      print(type(meta_batch))

In [ ]:
# task_loss, task_acc = [], []
# # special_grad = OrderedDict()  # Added for variants!
# #         coefs = {
# #             'crs_ent': 1,
# #             'clst': 0.8,
# #             'sep': -0.08,
# #             'l1': 1e-4,
# #             }
# for meta_batch in x:
#   support_set = meta_batch[: n_way * k_shot]  
#   query_set = meta_batch[n_way * k_shot :]    
  
#   fast_weights = OrderedDict(meta_model.named_parameters())
  
#   ### ---------- INNER TRAIN LOOP ---------- ###
#   # for inner_step in range(inner_train_step): 
#   train_label = create_label(n_way, k_shot).to(device)
#   logits = meta_model.functional_forward(support_set, fast_weights)
#   loss = loss_fn(logits, train_label)
#   task_loss.append(loss)
#   task_acc.append(calculate_accuracy(logits, train_label))

In [ ]:
train_loader = DataLoader(train_set,
                # The "batch_size" here is not \
                #    the meta batch size, but  \
                #    how many different        \
                #    characters in a task,     \
                #    i.e. the "n_way" in       \
                #    few-shot classification.
                batch_size=n_way,
                num_workers=4,
                shuffle=True,
                drop_last=True)

In [ ]:
for i, (image, label) in enumerate(train_loader):
  print(label)

In [ ]:
class MiniImageNet(Dataset):
  def __init__(self, data_dir,k_way,q_query):
    """Dataset class representing miniImageNet dataset

    # Arguments:
        subset: Whether the dataset represents the background or evaluation set
    """
    self.file_list = [f for f in glob.glob(
        data_dir+"*", 
        recursive=True)]
    self.n = k_way + q_query        

    normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                              std=(0.229, 0.224, 0.225))
    # Setup transforms
    self.transform = transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor(),
        normalize,
    ])

  def __getitem__(self, idx):
    sample = np.arange(600)

    # For random sampling the characters we want.
    np.random.shuffle(sample) 
    img_path = self.file_list[idx]
    img_list = [f for f in glob.glob(
        img_path + "**/*.jpg", recursive=True)]
    img_list.sort()
    imgs = [self.transform(
        Image.open(img_file)) 
        for img_file in img_list]
    # `k_way + q_query` examples for each character
    imgs = torch.stack(imgs)[sample[:self.n]] 
    lst = [img_path.split('/')[-1]]+[imgs]
    return lst

  def __len__(self):
    return len(self.file_list)

In [ ]:
train_data_path = '/content/miniImageNet/train/'
val_data_path = '/content/miniImageNet/val/'
test_data_path = '/content/miniImageNet/test'
# vgg16_bn_f = vgg16_bn_features(pretrained=True)
# print(vgg16_bn_f)
train_set = MiniImageNet(train_data_path, k_shot, q_query)
val_set = MiniImageNet(val_data_path,k_shot,q_query)
test_set = MiniImageNet(test_data_path, k_shot, q_query)

(train_loader, val_loader, test_loader), \
(train_iter, val_iter, test_iter) = dataloader_init(
                    (train_set, val_set, test_set),n_way)

In [ ]:
def get_meta_batch(meta_batch_size,
                   k_shot, q_query, 
                   data_loader, iterator):
  data = []
  label_lst =[]
  for _ in range(meta_batch_size):
    try:
      # a "task_data" tensor is representing \
      #     the data of a task, with size of \
      #     [n_way, k_shot+q_query, 1, 28, 28]
      task_data = iterator.next()  
    except StopIteration:
      iterator = iter(data_loader)
      task_data = iterator.next()
    label = task_data[0]
    label_lst.append(label)
    task_data = task_data[1]
    train_data = (task_data[:, :k_shot]
                  .reshape(-1, 3, 224, 224))
    val_data = (task_data[:, k_shot:]
                .reshape(-1, 3,224, 224))
    task_data = torch.cat(
        (train_data, val_data), 0)
    data.append(task_data)
  return torch.stack(data).to(device), iterator,label_lst

In [ ]:
# data = []
# for _ in range(meta_batch_size):
#     try:
#       # a "task_data" tensor is representing \
#       #     the data of a task, with size of \
#       #     [n_way, k_shot+q_query, 1, 28, 28]
#       task_data = train_iter.next()  
#     except StopIteration:
#       iterator = iter(train_loader)
#       task_data = iterator.next()
    
#     label = task_data[0]
#     task_data= task_data[1]
#     train_data = (task_data[:, :k_shot]
#                   .reshape(-1, 3, 224, 224))
#     # train_data = (task_data[:,1 :k_shot+1]
#     #               .reshape(-1, 3, 224, 224))
#     # print(train_data.shape())
#     # print(train_data.shape())
#     val_data = (task_data[:, k_shot:]
#                 .reshape(-1, 3,224, 224))
#     task_data = torch.cat(
#         (train_data, val_data), 0)
#     # data.append(task_data)

In [ ]:
x, train_iter,label_lst = get_meta_batch(
          meta_batch_size, k_shot, q_query, 
          train_loader, train_iter)

In [ ]:
np.array(label_lst).shape,x.shape

In [ ]:
label_lst

In [ ]:
label=label_lst[0]
list(label[:1])

In [ ]:
def labeltoindex(label):
  with open('/content/imagenet_index_class.json', 'r', encoding='utf8')as fp:
    labels_all = json.load(fp)
  index = labels_all[label]
  return index

In [ ]:
def find_label(label_lst,k_shot):
  
  labels = list(label_lst)
  index_lst=[]
  for label in labels:
    index = labeltoindex(label)
    index_lst=index_lst+[index]*k_shot
    # index_lst.append(labeltoindex(label))
  return torch.tensor(index_lst)

In [ ]:
val_label=find_label(label,15)

In [ ]:
# print(find_label(label,1,1),type(find_label(label,1,1)[0]))

In [ ]:
train_label = create_label(n_way, 15).to(device)
train_label

In [ ]:
res_lst=[]
for meta_batch in x:
      print(meta_batch.shape)
      # vgg16_conv(meta_batch)
      # support_set = meta_batch[: n_way * k_shot] 
      query_set = meta_batch[n_way * k_shot :] 
      # img = support_set[1:2,:,:,:]
      # print(img.shape)
      res= vgg16_conv(query_set)
      # res_lst.append(res)
      break
      # query_set = meta_batch[n_way * k_shot :]    
      # res=vgg16_conv(support_set)
      # fast_weights = OrderedDict(model.named_parameters())

In [ ]:
res_lst

In [ ]:
support_set.shape
for img in support_set:
  print(img.shape)

In [ ]:
for i, t in enumerate(train_loader):
  print(type(t))

In [ ]:
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
torch.argmax(res,-1).cpu().numpy()

In [ ]:
calculate_accuracy(res,val_label)

In [ ]:
loss_fn(res,val_label.to(device))

In [ ]:
def calculate_accuracy(logits, val_label):
  """ utility function for accuracy calculation """
  torch.argmax(logits,)
  acc = np.asarray([(
      torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy())]
      ).mean() 
  return acc

In [ ]:
vgg16_conv = Vgg16Conv()
vgg16_conv.eval()
vgg16_conv.to(device)